# MSA 2023 Phase 2 - Part 3 (Example)

In this example notebook, the well-known Iris dataset is used to build an XGBoost classifier, which is then deployed onto Azure via the the Azure Machine Learning Software Development Kit (SDK).

In [ ]:
# Load XGBoost classifier
from xgboost import XGBClassifier

# Load Iris dataset
from sklearn import datasets
iris = datasets.load_iris()

# Load Azure Machine Learning SDK core packages and modules
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Load other necessary packages and modules
from sklearn.model_selection import train_test_split
import numpy as np
import requests, json, os

### Train model

In [5]:
# Split data into a training set and a test set (not used in this example)
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create XGBoost classifier and save model
xgbClf = XGBClassifier(use_label_encoder = False)
xgbClf.fit(X_train, y_train)
xgbClf.save_model("model.json")

[18:54:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


### Connect to workspace

In [9]:
# Load and connect to workspace using the config.json file saved from step 5 of getting-started-with-azure-ml.md 
ws = Workspace.from_config(path="/path/to/config.json")

# A web browser window should open where you have to sign-in to Azure. If you connect to your 
# workspace successfully, you should see an output beginning with "Workspace.create(...)" being printed.
print(ws)

Performing interactive authentication. Please follow the instructions on the terminal.
Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace.create(name='MSA-ModelDeployment', subscription_id='1728fb6f-0674-4adc-b1a8-0462995e20a5', resource_group='MSA-ModelDeployment')


### Register model

In [10]:
# Register saved model with an appropriate model name
model = Model.register(ws, model_name = "iris-xgboost", model_path = "/path/to/model.json")

Registering model iris-xgboost


### Create entry script

Registering a model only uploads it onto the cloud, there is no code that can allow us to interface with it. As such, an entry script needs to be created that will run when the model receives data. This script contains two functions:

1. ```init()```: loads the model via a global variable
2. ```run()```: loads the input data, runs the model on it, and returns its predictions to the client

For the Iris dataset, a script called echo_score.py has been written below.

In [4]:
# echo_score.py

# Since the model works with label-encoded data, we can create a dictionary to get the actual class names
classes = {0: "setosa", 1: "versicolor", 2: "virginica"}

def init():
    global model
    model = XGBClassifier(use_label_encoder=False)
    model.load_model(os.path.join(os.getenv("AZUREML_MODEL_DIR"), "/path/to/model.json"))

def run(request):
    data = json.loads(request)
    data = np.array(data["data"])
    response = model.predict(data)
    return [classes.get(key) for key in response]

### Create remote virtual environment

After registering our model and creating an entry script, we need to create a remote virtual environment. This environment should have exactly the same modules (including their versions) as the virtual environment on our local machine where we know our model runs.

Under the hood, Azure is creating a Docker container which is essentially a more isolated version of a virtual machine (you don't need to know the specifics of Docker to complete this part). This container will contain the remote virtual environment we want our model to run inside.

There are two options to create a remote virtual environment, as shown below. Using the first option is the easiest but may result in deployment errors, in which case try the second option.

In [12]:
# OPTION 1: Create from Pip requirements
# env = Environment.from_pip_requirements(name="iris-xgboost", file_path="/path/to/requirements.txt")

# OPTION 2: Create from scratch by manually adding required packages
env = Environment(name="iris-xgboost")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("xgboost")
env.python.conda_dependencies = conda_dep

### Create inference configuration

An inference configuration contains all the configuration settings needed for your remote virtual environment.

In [ ]:
inference_config = InferenceConfig(
    environment=env,
    source_directory="/path/to/source_directory",
    entry_script="path/to/echo_score.py",
)

### Deploy model

In [13]:
# Change the amount of CPU or memory depending on what kind of model used
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service = Model.deploy(
    ws,
    "iris-xgboost",
    [model],
    inference_config,
    aci_config,
    overwrite=True,
)

# Once the model has been deployed, it will appear in the Endpoints section of the Azure Machine Learning Studio
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-26 18:59:24+12:00 Creating Container Registry if not exists..
2021-07-26 18:59:39+12:00 Registering the environment..
2021-07-26 18:59:42+12:00 Building image..
2021-07-26 19:10:28+12:00 Generating deployment configuration.
2021-07-26 19:10:29+12:00 Submitting deployment to compute.
2021-07-26 19:10:39+12:00 Checking the status of deployment iris-xgboost.
2021-07-26 19:13:37+12:00 Checking the status of inference endpoint iris-xgboost.
Succeeded
ACI service creation operation finished, operation "Succeeded"


### Test model endpoint

Our deployed model can be called via a REST API. If it returns a prediction, then we have successfully deployed our model!

In [16]:
uri = service.scoring_uri

# Your API endpoint will look like this: http://16a75b9c-e6cc-47cd-89f6-215e077c43a9.australiaeast.azurecontainer.io/score
requests.get("<Add your API endpoint here>")
headers = {"Content-Type": "application/json"}
data = {
    "data": [[6.1, 2.8, 4.7, 1.2]],
}
data = json.dumps(data)

# Output a classification/regression result back to the user
response = requests.post(uri, data=data, headers=headers)
print(response.json())

['versicolor']


### Clean up

Stop the compute instance that contains your deployed model when not in use (to avoid incurring unnecessary costs) by going to the Compute section of the Azure Machine Learning Studio and pressing Stop on the compute instance of your model. 

If you want to use the model again, press Start.